<a href="https://colab.research.google.com/github/vg11072001/llm_with_HumanFeedback/blob/main/baseimage_for_llm_finetuing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

lmsys_chatbot_arena_path = kagglehub.competition_download('lmsys-chatbot-arena')

print('Data source import complete.')


100%|██████████| 57.0M/57.0M [00:00<00:00, 95.6MB/s]

Extracting files...


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets
!pip install kaggle_secrets

ERROR: Could not find a version that satisfies the requirement kaggle_secrets (from versions: none)
ERROR: No matching distribution found for kaggle_secrets


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

import json

from tqdm.auto import tqdm
from datasets import load_dataset
from huggingface_hub import login
try:
  from kaggle_secrets import UserSecretsClient
  user_secrets = UserSecretsClient()
  secret_value_0 = user_secrets.get_secret("HF_TOKEN")
except:
  pass

try:
  from google.colab import userdata
  secret_value_0 = userdata.get("HF_TOKEN")
except:
  pass

login(secret_value_0)

In [ ]:
lmsys_chatbot_arena_path

'/root/.cache/kagglehub/competitions/lmsys-chatbot-arena'

In [ ]:
train = pd.read_csv('/root/.cache/kagglehub/competitions/lmsys-chatbot-arena/train.csv')
test = pd.read_csv('/root/.cache/kagglehub/competitions/lmsys-chatbot-arena/test.csv')

**Data preparation**

In [ ]:
print('-----------train-----------')
print(train.head(1))
print(train.shape)
print(train.columns)

-----------train-----------
      id             model_a     model_b  \
0  30192  gpt-4-1106-preview  gpt-4-0613   

                                              prompt  \
0  ["Is it morally right to try to have a certain...   

                                          response_a  \
0  ["The question of whether it is morally right ...   

                                          response_b  winner_model_a  \
0  ["As an AI, I don't have personal beliefs or o...               1   

   winner_model_b  winner_tie  
0               0           0  
(57477, 9)
Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [ ]:
skf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=0)
group_id = train['prompt']
label_id = train['winner_model_a winner_model_b winner_tie'.split()].values.argmax(1)
splits = list(skf.split(train, label_id, group_id))

In [ ]:
print(f"group_id: {group_id}")
print(f"label: {label_id}, len {len(label_id)}")

group_id: 0        ["Is it morally right to try to have a certain...
1        ["What is the difference between marriage lice...
2        ["explain function calling. how would you call...
3        ["How can I create a test set for a very rare ...
4        ["What is the best way to travel from Tel-Aviv...
                               ...                        
57472    ["A simple mnemonic for \u03c0:\n\"How I wish ...
57473    ["In python, implement a naive Bayes with gaus...
57474    ["is it unethical to work on building weapons?...
57475    ["If a bait contains 0,0025% bromadiolon then ...
57476    ["three kids eat three apples in three days, h...
Name: prompt, Length: 57477, dtype: object
label: [0 1 2 ... 0 1 0], len 57477


In [ ]:
splits

[(array([    0,     1,     2, ..., 57474, 57475, 57476]),
  array([    3,    21,    33, ..., 57467, 57468, 57470])),
 (array([    0,     1,     2, ..., 57474, 57475, 57476]),
  array([    4,     6,    12, ..., 57456, 57460, 57464])),
 (array([    0,     3,     4, ..., 57474, 57475, 57476]),
  array([    1,     2,    10, ..., 57455, 57459, 57461])),
 (array([    0,     1,     2, ..., 57468, 57469, 57470]),
  array([    7,     8,    13, ..., 57474, 57475, 57476])),
 (array([    1,     2,     3, ..., 57474, 57475, 57476]),
  array([    0,     5,     9, ..., 57457, 57463, 57469]))]

In [ ]:
print(f"number of splits {len(splits)}, each splits contain {len(splits[0])}, and within it {len(splits[0][0])}: {len(splits[0][1])}, ideally {len(train)//5}")

number of splits 5, each splits contain 2, and within it 46031: 11446, ideally 11495


In [ ]:
splits[0][1]

array([    3,    21,    33, ..., 57467, 57468, 57470])

In [ ]:
group_id

,prompt
0,"[""Is it morally right to try to have a certain..."
1,"[""What is the difference between marriage lice..."
2,"[""explain function calling. how would you call..."
3,"[""How can I create a test set for a very rare ..."
4,"[""What is the best way to travel from Tel-Aviv..."
...,...
57472,"[""A simple mnemonic for \u03c0:\n\""How I wish ..."
57473,"[""In python, implement a naive Bayes with gaus..."
57474,"[""is it unethical to work on building weapons?..."
57475,"[""If a bait contains 0,0025% bromadiolon then ..."


In [ ]:
label_id

array([0, 1, 2, ..., 0, 1, 0])

In [ ]:
print(len(group_id.unique()))

51734


In [ ]:
train["fold"] = -1
for fold, (_, valid_idx) in enumerate(splits):
    train.loc[valid_idx, "fold"] = fold

train.head(1)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,fold
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,4


In [ ]:
train['fold'].value_counts()

In [ ]:
tqdm.pandas()

train_original = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
external_data = load_dataset("lmsys/chatbot_arena_conversations")["train"].to_pandas()

print('-----------external_data-----------')
print(external_data.head(1))
print(external_data.shape)
print(external_data.columns)

In [ ]:
def separate_conv(conv):
    try:
        user_texts = [x["content"] for x in conv if x["role"] == "user"]
        assistant_texts = [x["content"] for x in conv if x["role"] == "assistant"]

        return user_texts, json.dumps(assistant_texts)
    except:
        print(conv)

external_data["prompt_a"], external_data["response_a"] = zip(*external_data.conversation_a.progress_apply(separate_conv))
external_data["prompt_b"], external_data["response_b"] = zip(*external_data.conversation_b.progress_apply(separate_conv))
external_data["prompt"] = external_data["prompt_a"].progress_apply(json.dumps)
print(external_data.winner.value_counts())

def one_hot_encode(winner):
    return pd.Series([int("model_a" == winner), int("model_b" == winner), int("tie" == winner or "tie (bothbad)" == winner)])

external_data[["winner_model_a", "winner_model_b", "winner_tie"]] = (external_data.winner.progress_apply(one_hot_encode))
cols = ["question_id","model_a","model_b","prompt","response_a","response_b","winner_model_a","winner_model_b","winner_tie"]
external_data = pd.DataFrame(external_data[cols].copy().values, columns=train_original.columns)

superset = pd.concat([external_data, train_original]).reset_index(drop=True)
external_data_deduplicated = superset.drop_duplicates(subset=["prompt"], keep="last")
external_data_deduplicated = external_data_deduplicated[external_data_deduplicated.index.isin(external_data.index)]
print(len(external_data_deduplicated))
print(external_data_deduplicated.head(1))

# external_data.to_csv("output/lmsys-33k.csv", index=False)
external_data_deduplicated = external_data_deduplicated.reset_index(drop=True)
# external_data_deduplicated.to_csv("output/lmsys-33k-deduplicated.csv", index=False)

In [ ]:
print('-----------external_data_deduplicated-----------')
print(external_data_deduplicated.head(1))
print(external_data_deduplicated.shape)
print(external_data_deduplicated.columns)

print('-----------train-----------')
print(train.head(1))
print(train.shape)
print(train.columns)

**Config file**
- dataset building

**Tokenizer**

```bash
GemmaTokenizerFast(
        name_or_path='google/gemma-2-9b-it',
        vocab_size=256000,
        model_max_length=1000000000000000019884624838656,
        is_fast=True,
        padding_side='left',
        truncation_side='right',
        special_tokens={'bos_token': '<bos>',
                        'eos_token': '<eos>',
                        'unk_token': '<unk>',
                        'pad_token': '<pad>',
                        'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>']
                        },
        clean_up_tokenization_spaces=False,
        added_tokens_decoder = {
                        	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
                        	1: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
                        	2: AddedToken("<bos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
                            ...
                            ...
                            ...
                            ...
                            ...
                            255998: AddedToken("			   		", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
                        	255999: AddedToken("<unused99>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
                            }
                )
```

- SentencePiece Tokenizer: Used for subword tokenization and byte-level encoding.
- Special Tokens: Added to structure the input and output sequences.
- Tokenization Steps: Subword splitting, byte-level encoding, and adding special tokens.
- Output: A list of token IDs suitable for input into the model.
- This tokenization process ensures that the model can handle a wide range of text inputs efficiently and effectively, making it suitable for various natural language processing task.


1. **SentencePiece Tokenizer**

Gemma models use the SentencePiece tokenizer, which is a subword tokenizer. This tokenizer splits text into subwords, which helps in handling out-of-vocabulary (OOV) words and improves the efficiency of the model.
Key Features:
- Byte-Level Encoding: SentencePiece uses byte-level encoding, which means it can handle any Unicode text.
- Subword Tokenization: It splits words into subwords, which helps in managing rare words and unseen words during training.
- Preserved Whitespace: It preserves whitespace, which is important for maintaining the structure of the text.
- Trained on multilingual data.
- Uses ▁ to denote word boundaries.

2. **Tokenization Process**
Step-by-Step Tokenization:
- Text Input: The raw text is provided to the tokenizer.
- Subword Splitting: The SentencePiece tokenizer splits the text into subwords.
- Byte-Level Encoding: Each subword is encoded into bytes.
- Special Tokens: Special tokens like <bos>, <eos>, <pad>, and others are added to the tokenized text to mark the beginning and end of sequences and to handle padding.

3. **Special Tokens**

Gemma models use several special tokens to structure the input and output sequences:
- `<bos>`: Beginning of Sequence token.
- `<eos>`: End of Sequence token.
- `<pad>`: Padding token.
- `<start_of_turn>`: Indicates the start of a turn in a multi-turn conversation.
- `<end_of_turn>`: Indicates the end of a turn in a multi-turn conversation.

These special tokens help the model understand the structure of the input and generate appropriate responses.


3. Key Differences

| Feature           | BERT       | Gemma 2    | Llama 3    |
|-------------------|------------|------------|------------|
| Algorithm         | WordPiece  | SentencePiece | BPE        |
| Vocabulary Size   | ~30k       | ~256k      | 128k       |
| Special Tokens    | [CLS], [SEP] | `<bos>`, `<eos>` | `<s>, </s>`  |
| Case Handling     | Optional   | Case-sensitive | Case-sensitive |
| Whitespace        | Ignored    | ▁ markers  | Explicit spaces |
| Emoji Handling    | Poor       | Good       | Excellent |
| Multilingual      | No         | Yes        | Yes        |


**pretrained_model_functionality from transformer**
- https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/model#transformers.PreTrainedModel
- about gemma2: https://huggingface.co/docs/transformers/en/model_doc/gemma2

In [ ]:
# testing of tokenizer process
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer_testing = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")

# Example text
text_testing = "This is an example sentence model path when have acces to dowload weights from HF"
max_length1 = 10
# tokenizer(text, add_special_tokens=False, max_length=max_length, truncation=True).input_ids

# Tokenize the text
inputs_testing = tokenizer_testing(text_testing, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs_testing1 = tokenizer_testing(text_testing, add_special_tokens=False, max_length=max_length1, truncation=True)

# Print the tokenized input
print(inputs_testing)
print(inputs_testing1)
print(len(text_testing))
print(len(inputs_testing1.input_ids))
# print(len(inputs_testing['input_ids'][0]))
# print(len(inputs_testing['attention_mask']))

In [ ]:
from transformers import AutoTokenizer

model_name_or_path = "google/gemma-2-9b-it" # model path when have acces to dowload weights from HF
tokenizer1 = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer1

**Dataset Preparation**

Using Map style dataset for this:

![image.png](attachment:cce76ede-fdfb-4702-9764-76ed3efec276.png)

Other type is iterable.

In [ ]:
idx = 1
data1 = train

data = data1.iloc[idx].to_dict()

print(f"data: {data}\n\n")

prompts = json.loads(data["prompt"])
responses_a = json.loads(data["response_a"])
responses_b = json.loads(data["response_b"]),

In [ ]:
import torch
max_length1 = 4096
support_system_role1 = False

tokenizer = tokenizer1
max_length = max_length1
support_system_role = support_system_role1

PROMPT_PREFIX = """Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie."""

PROMPT_SUFFIX = "verdict is: [["

LABEL_COLS = ["winner_model_a", "winner_model_b", "winner_tie"]

head = "<|The Start of Conversation between a User and two Assistants|>"
tail = "<|The End of Conversation between a User and two Assistants|>\n"
parts = []

for prompt, response_a, response_b in zip(prompts, responses_a, responses_b):
    if prompt is None:
        prompt = "null"
    if response_a is None:
        response_a = "null"
    if response_b is None:
        response_b = "null"
    parts.append(f"\n### User:\n{prompt}\n\n### Assistant A:\n{response_a}\n\n### Assistant B:\n{response_b}\n")

text = "".join(parts)
print(text)
print(tokenizer(text, add_special_tokens=False, max_length=max_length, truncation=True))
input_ids = tokenizer(text, add_special_tokens=False, max_length=max_length, truncation=True).input_ids
truncated_text = tokenizer.decode(input_ids)
# print(truncated_text)

conversation = head + truncated_text + tail
print(f"input_ids length with truncation: {len(input_ids)}\n\n")
print(f"truncated_text length: {len(truncated_text.split())}\n\n")
print(f"conversation: {conversation}\n\n")
print(f"conversation words length: {len(conversation.split())}\n\n")

if support_system_role:
    messages = [{"role": "system", "content": PROMPT_PREFIX}, {"role": "user", "content": conversation}]
else:
    messages = [{"role": "user", "content": f"{PROMPT_PREFIX}\n{conversation}"}]

input_text = (tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) + PROMPT_SUFFIX)
input_ids = tokenizer(input_text, add_special_tokens=False, return_tensors="pt").input_ids[0]
label = torch.tensor([data[col] for col in LABEL_COLS]).float()

print(f"after apply_chat_template input_ids length: {len(input_ids)}\n\n")
print(f"after apply_chat_template input_text words length: {len(input_text.split())}\n\n")
print(f"label length: {len(label)}\n\n")

# import numpy as np
# np.set_printoptions(threshold=np.inf)
# print(input_ids.numpy())

processorPAB = dict(input_ids=input_ids, input_text=input_text, label=label)

print(f"\n\nprocessorPAB: {processorPAB}")

In [ ]:
import torch
max_length1 = 4096 // 2
support_system_role1 = False

tokenizer = tokenizer1
max_length = max_length1
support_system_role = support_system_role1

head = "<|The Start of Assistant A’s Conversation with User|>"
sep = "<|The End of Assistant A’s Conversation with User|>\n\n<|The Start of Assistant B’s Conversation with User|>"
tail = "<|The End of Assistant B’s Conversation with User|>\n"
parts_a = []
parts_b = []

for prompt, response_a, response_b in zip(prompts, responses_a, responses_b):
    if prompt is None:
        prompt = "null"
    if response_a is None:
        response_a = "null"
    if response_b is None:
        response_b = "null"
    parts_a.append(f"\n### User:\n{prompt}\n\n### Assistant A:\n{response_a}\n")
    parts_b.append(f"\n### User:\n{prompt}\n\n### Assistant B:\n{response_b}\n")

text_a = "".join(parts_a)
text_b = "".join(parts_b)

input_ids_a = tokenizer(text_a, add_special_tokens=False, max_length=max_length, truncation=True).input_ids
input_ids_b = tokenizer(text_b, add_special_tokens=False, max_length=max_length, truncation=True).input_ids

truncated_text_a = tokenizer.decode(input_ids_a)
truncated_text_b = tokenizer.decode(input_ids_b)

conversation = head + truncated_text_a + sep + truncated_text_b + tail
print(f"conversation: {conversation}\n\n")

if support_system_role:
    messages = [{"role": "system", "content": PROMPT_PREFIX}, {"role": "user", "content": conversation}]
else:
    messages = [{"role": "user", "content": f"{PROMPT_PREFIX}\n{conversation}"}]

input_text = (tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) + PROMPT_SUFFIX)
input_ids = tokenizer(input_text, add_special_tokens=False, return_tensors="pt").input_ids[0]
label = torch.tensor([data[col] for col in LABEL_COLS]).float()

ProcessorPAPB = dict(input_ids=input_ids, input_text=input_text, label=label)

print(f"ProcessorPAPB: {ProcessorPAPB}")

In [ ]:
# processor = [ProcessorPAB, ProcessorPAPB]

import pandas as pd
import copy

# Define the processors
class ProcessorPAB:
    def __init__(self, tokenizer, max_length, support_system_role):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.support_system_role = support_system_role

    def build_input(self, data):
        # Example processing (you need to implement the actual processing logic)
        return {
            'input_ids': self.tokenizer.encode(data['prompt'], max_length=self.max_length, truncation=True),
            'labels': data['winner_model_a']
        }

class ProcessorPAPB:
    def __init__(self, tokenizer, max_length, support_system_role):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.support_system_role = support_system_role

    def build_input(self, data):
        # Example processing (you need to implement the actual processing logic)
        return {
            'input_ids': self.tokenizer.encode(data['prompt'], max_length=self.max_length, truncation=True),
            'labels': data['winner_model_b']
        }

# Initialize processors
processor = [
    ProcessorPAB(tokenizer=tokenizer, max_length=max_length, support_system_role=False),
    ProcessorPAPB(tokenizer=tokenizer, max_length=max_length // 2, support_system_role=False)
]

# Load the DataFrame
# df = pd.read_csv("../output/dtrainval.csv")
df = train
include_swap = True

# Apply the query
query = f"fold != {fold}"
df = df.query(query).reset_index(drop=True)

# Process each sample
dataset0 = []
for idx in range(len(df[0:20])):
    data = df.iloc[idx].to_dict()
    ret = [proc.build_input(data) for proc in processor]

    if include_swap:
        data_swap = copy.deepcopy(data)
        data_swap["model_a"], data_swap["model_b"] = data_swap["model_b"], data_swap["model_a"]
        data_swap["response_a"], data_swap["response_b"] = data_swap["response_b"], data_swap["response_a"]
        data_swap["winner_model_a"], data_swap["winner_model_b"] = data_swap["winner_model_b"], data_swap["winner_model_a"]
        ret.extend([proc.build_input(data_swap) for proc in processor])

    dataset0.append(ret)

# Now dataset0 contains the processed data
dataset0

In [ ]:
# Initialize processors
processor = [
    ProcessorPAB(tokenizer=tokenizer, max_length=max_length, support_system_role=False),
    ProcessorPAPB(tokenizer=tokenizer, max_length=max_length // 2, support_system_role=False)
]

# Load the DataFrame
# df = pd.read_csv("../output/dtrainval.csv")
df = external_data
include_swap = True

# Apply the query
query = f"fold != {fold}"
# df = df.query(query).reset_index(drop=True)

# Process each sample
dataset1 = []
for idx in range(len(df[0:20])):
    data = df.iloc[idx].to_dict()
    ret = [proc.build_input(data) for proc in processor]

    if include_swap:
        data_swap = copy.deepcopy(data)
        data_swap["model_a"], data_swap["model_b"] = data_swap["model_b"], data_swap["model_a"]
        data_swap["response_a"], data_swap["response_b"] = data_swap["response_b"], data_swap["response_a"]
        data_swap["winner_model_a"], data_swap["winner_model_b"] = data_swap["winner_model_b"], data_swap["winner_model_a"]
        ret.extend([proc.build_input(data_swap) for proc in processor])

    dataset1.append(ret)

# Now dataset1 contains the processed data
dataset1

In [ ]:
print(type(dataset0[0][0].keys()))
print(len(dataset0[0][0]['input_ids']))
print(len(dataset1[0][0]['input_ids']))

In [ ]:
dataset = torch.utils.data.ConcatDataset([dataset0, dataset1])
dataset

**Data Laoder**

![image.png](attachment:4160729e-b7c9-403c-85b3-4e7bfeacac84.png)


To get the `DataLoader` we first need data samples collating to batch , and all other arugments like `batch_size`, `drop_last`, `batch_sampler`.

> When automatic batching is enabled, collate_fn is called with a list of data samples at each time. It is expected to collate the input samples into a batch for yielding from the data loader iterator. The rest of this section describes the behavior of the default collate_fn (default_collate()).


In PyTorch's `DataLoader`, the collate function is applied after the individual samples are fetched from the dataset and grouped into a batch (as a list of samples). Here's the step-by-step process:

1. **Initialize the Dataset and DataLoader:**
The DataLoader loads a map-style dataset (e.g., torch.utils.data.Dataset), which provides access to individual samples by index.

2. **Sampling:**
The DataLoader uses a sampler (e.g., `RandomSampler`) to select indices from the dataset. These indices are grouped into batches by a batch sampler.

3. **Fetch Samples:**
For each batch of indices, the DataLoader fetches the corresponding individual samples from the dataset (e.g., dataset[0], dataset[1], etc.) and collects them into a list (e.g., `[sample0, sample1, sample2]`).

4. **Collate Function:**
The collate function (`collate_fn`) is then applied to this list of samples to merge them into a batched structure (e.g., padding sequences, stacking tensors into a batch dimension). The collate function is responsible for converting the list of samples into a batch format suitable for the model (e.g., `torch.stack()` for fixed-size tensors or `pad_sequence()` for variable-length sequences).
Example Workflow:

In [ ]:
# torch.utils.data.DataLoader(dataset,
#         shuffle=training,
#         batch_size=batch_size,
#         num_workers=num_workers,
#         drop_last=training,
#         collate_fn=ShardedMaxTokensCollator(
#             max_tokens=max_tokens,
#             base_collator=VarlenCollator(),
#             sort_samples=training,
#         ),
#     )

In [ ]:
batch_size =80
num_workers=4
training=True

max_tokens = 1024 * 16

In [ ]:
import os
import torch
import torch.distributed as dist

# Manually set environment variables
os.environ['RANK'] = '1'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

# Initialize the distributed environment
dist.init_process_group(backend='nccl', init_method='tcp://localhost:12345', rank=0, world_size=1)

# Get the rank and world size
rank = dist.get_rank()
world_size = dist.get_world_size()

1. `world_size`:
The total number of processes (e.g., GPUs or machines) participating in distributed training.
Role: Determines how data is partitioned into shards, ensuring each process handles an equal portion of the dataset.

2. `rank`:
A unique identifier for each process within a distributed setup `(ranges from 0 to world_size - 1)`.
Role: Identifies which shard (data partition) a process should work on. For example, `rank=2` in a `world_size=4` setup handles the third data shard.

3. `shard`:
A logical division of the dataset assigned to a specific process (identified by `rank`).
Role: Enables parallel processing by ensuring each process only handles its own shard, reducing memory and computation overhead.

1. **Sharding the Dataset**:
```python
sample_index_matrix = torch.arange(len(samples)).reshape(-1, self.world_size)
size_matrix = torch.tensor([sample["input_ids"].size(0) for sample in samples]).reshape(-1, self.world_size)
```
- `sample_index_matrix`: Reshapes indices of all samples into a matrix with world_size columns. Each column represents a shard (e.g., world_size=2 → `n x 2` matrix).
- `size_matrix`: Corresponding token counts for each sample, reshaped to align with sample_index_matrix.

2. **Token-Aware Micro-Batch Splitting**:
```python
micro_batch_segments = []
start = 0
for end in range(size_matrix.size(0)):
    if (size_matrix[start : end + 1].sum(0) > self.max_tokens).any() and end > start:
        micro_batch_segments.append((start, end))
        start = end
if start < size_matrix.size(0):
    micro_batch_segments.append((start, size_matrix.size(0)))
```
- **Greedy Chunking**: Segments the token data (size_matrix) into chunks (rows) where each chunk's cumulative tokens per shard (column) ≤ `self.max_tokens`.
- **Edge Handling**: Ensures the last segment is included even if it’s smaller than the max token limit.

3. **Extracting Micro-Batches for Each Process**:
```python
micro_batches = []
for start, end in micro_batch_segments:
    micro_batch_samples = [samples[i] for i in sample_index_matrix[start:end, self.rank]]
    micro_batches.extend(self.base_collator(micro_batch_samples))
```
- **Per-Rank Selection**: For each segment, selects samples from the current process’s shard (column) using sample_index_matrix `[start:end, self.rank]`.
- **Collation**: Preprocesses batches (e.g., padding tokens) for model compatibility via `self.base_collator`.

In [ ]:
import torch

samples = [
{"input_ids":  torch.tensor([3, 4]), "label": 1},
{"input_ids": torch.tensor([5, 6, 7, 8]), "label": 0},
{"input_ids": torch.tensor([9, 10, 11]), "label": 1},
{"input_ids": torch.tensor([12, 13]), "label": 0},
{"input_ids": torch.tensor([14, 15, 16, 17, 18]), "label": 1}
]
sort_samples = True
max_tokens = 10
world_size = 2 # (2 processes)
rank = 0 # then 1 on paralllel one

print(f"Rank: {rank}, World Size: {world_size}\n\n")


# flatten if a 'sample' is a list of samples
if isinstance(samples[0], list):
    samples = [sample for s in samples for sample in s]
print(f'flatten samples {samples}\n\n')

# sorting on basis of size of input_ids within sample
if sort_samples:
    samples = sorted(samples, key=lambda x: x["input_ids"].size(0))
print(f'sort_samples {samples}\n\n')

# to handle future split samples into shards with padding till following conditions is true
while len(samples) % world_size != 0:
    samples.append(samples[-1])
    print(f"Padding samples to make them divisible by num_shards={world_size}")
print(f'padding {samples}\n\n')

sample_index_matrix = torch.arange(len(samples)).reshape(-1, world_size)
size_matrix = torch.tensor([sample["input_ids"].size(0) for sample in samples]).reshape(-1, world_size)
micro_batch_segments = []
# (start, end) that (size_matrix[start:end].sum(0) <= max_tokens).all()
print(f'sample_index_matrix {sample_index_matrix}\n\n')
print(f'size_matrix {size_matrix}\n\n')

start = 0
for end in range(size_matrix.size(0)):
    # look ahead
    if (size_matrix[start : end + 1].sum(0) > max_tokens).any() and end > start:
        micro_batch_segments.append((start, end))
        start = end
print(f'micro_batch_segments {micro_batch_segments}\n\n')

if start < size_matrix.size(0):
    micro_batch_segments.append((start, size_matrix.size(0)))
print(f'micro_batch_segments {micro_batch_segments}\n\n')

micro_batches = []
for start, end in micro_batch_segments:
    micro_batch_samples = [samples[i] for i in sample_index_matrix[start:end, rank]]
    print(f'{start}, {end}, {micro_batch_samples}')
    # samples = micro_batch_samples
    seq_lens = []
    cu_seqlens = [0]
    end = 0
    input_idss = []
    position_idss = []

    for sample in micro_batch_samples:
        seq_len = sample["input_ids"].size(0)
        seq_lens.append(seq_len)
        end += seq_len
        cu_seqlens.append(end)
        input_idss.append(sample["input_ids"])
        position_idss.append(torch.arange(seq_len))

    input_ids = torch.cat(input_idss, dim=0)
    position_ids = torch.cat(position_idss, dim=0)

    data = dict(batch_size=len(micro_batch_samples),
        input_ids=input_ids.unsqueeze(0),
        position_ids=position_ids.unsqueeze(0),
        seq_lens=seq_lens,
        cu_seqlens=torch.tensor(cu_seqlens, dtype=torch.int32),
        max_seq_len=max(seq_lens))
    print(f"micro_batch_samples, {micro_batch_samples}\n")
    print([sample["label"] for sample in micro_batch_samples])
    # data["label"] = torch.stack([sample["label"] for sample in micro_batch_samples], dim=0)
    data["label"] = torch.stack([torch.tensor(sample["label"]) for sample in micro_batch_samples], dim=0)


    for key in micro_batch_samples[0]:
        if key not in data.keys():
            data[key] = [sample[key] for sample in micro_batch_samples]
    micro_batches.extend([data])
    print(f'micro_batches {micro_batches}\n\n')

print(f'micro_batch_samples {micro_batch_samples}\n\n')
print(f'micro_batches {micro_batches}\n\n')

# Clean up
# dist.destroy_process_group()

* `max_tokens`: specifies the maximum number of tokens allowed in a single batch.

* `max_seq_len or max_length`: specifies the maximum length of a single sequence.

In [ ]:
VAL_FOLD = 0
max_tokens = 1024 * 16

max_length = 4096

From text.csv i found following micro_batch!

```markdown
{
'batch_size': 3,
'input_ids': tensor([[     2,    106,   1645,  ...,    603, 235292,  15695]]),
'position_ids': tensor([[   0,    1,    2,  ..., 1948, 1949, 1950]]),
'seq_lens': [298, 687, 1951],
'cu_seqlens': tensor([   0,  298,  985, 2936], dtype=torch.int32),
'max_seq_len': 1951,
'label': tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),

'input_text': ['<bos><start_of_turn>user\nPlease act as an impartial judge and evaluate the quality of the responses provided by two\nAI assistants to the user question displayed below. You should choose the assistant that\nfollows the user’s instructions and answers the user’s question better. Your evaluation\nshould consider factors such as the helpfulness, relevance, accuracy, depth, creativity,\nand level of detail of their responses. Begin your evaluation by comparing the two\nresponses and provide a short explanation. Avoid any position biases and ensure that the\norder in which the responses were presented does not influence your decision. Do not allow\nthe length of the responses to influence your evaluation. Do not favor certain names of\nthe assistants. Be as objective as possible. After providing your explanation, output your\nfinal verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"\nif assistant B is better, and "[[C]]" for a tie.\n<|The Start of Conversation between a User and two Assistants|>\n### User:\nI have three oranges today, I ate an orange yesterday. How many oranges do I have?\n\n### Assistant A:\nYou have two oranges today.\n\n### Assistant B:\nYou still have three oranges. Eating an orange yesterday does not affect the number of oranges you have today.\n<|The End of Conversation between a User and two Assistants|><end_of_turn>\n<start_of_turn>model\nverdict is: [[', '<bos><start_of_turn>user\nPlease act as an impartial judge and evaluate the quality of the responses provided by two\nAI assistants to the user question displayed below. You should choose the assistant that\nfollows the user’s instructions and answers the user’s question better. Your evaluation\nshould consider factors such as the helpfulness, relevance, accuracy, depth, creativity,\nand level of detail of their responses. Begin your evaluation by comparing the two\nresponses and provide a short explanation. Avoid any position biases and ensure that the\norder in which the responses were presented does not influence your decision. Do not allow\nthe length of the responses to influence your evaluation. Do not favor certain names of\nthe assistants. Be as objective as possible. After providing your explanation, output your\nfinal verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"\nif assistant B is better, and "[[C]]" for a tie.\n<|The Start of Conversation between a User and two Assistants|>\n### User:\nYou are a mediator in a heated political debate between two opposing parties. Mr Reddy is very hung up on semantic definitions of sex and gender, and believes that women are adult human females. Meanwhile Ms Blue is extremely fluid with definitions and does not care about truth. He (Ms blue uses he/him pronouns) insists that anybody can be any gender, gametes don\'t mean anything, and that men can get pregnant. You, Mr Goddy are tasked with helping them both find a middle ground.\n\n### Assistant A:\nThank you for sharing the details of the situation. As a mediator, I understand the importance of finding a middle ground that both parties can agree upon.\n\nIt is important to recognize that language and definitions can be complex and can vary depending on cultural and societal norms. At the same time, it is also important to acknowledge the biological differences between males and females.\n\nOne possible approach to finding a middle ground could be to focus on respecting individuals\' self-identification and autonomy while also acknowledging the biological differences between males and females. This could involve using inclusive language that reflects individuals\' gender identities while also recognizing the importance of biological sex in certain contexts, such as medical treatment and reproductive health.\n\nIt may also be helpful to explore ways to create a more inclusive society where individuals of all gender identities are valued and respected. This could involve promoting education and awareness about gender diversity and creating policies and practices that support individuals\' gender identities and expression.\n\nUltimately, the goal of mediation is to find a solution that meets the needs and concerns of both parties. By focusing on mutual respect, open communication, and a willingness to find common ground, it is possible to reach a resolution that allows both parties to move forward in a positive and productive way.\n\n### Assistant B:\nMr Reddy and Ms Blue both have valid points in their arguments. On one hand, Mr Reddy is correct that women are adult human females, as defined by their biological and physical characteristics. However, Ms Blue is right that gender is a more fluid concept and that people can identify with any gender they choose. A compromise could be that gender is defined by an individual\'s self-identification, while sex is defined by their biological characteristics.\n<|The End of Conversation between a User and two Assistants|><end_of_turn>\n<start_of_turn>model\nverdict is: [[', '<bos><start_of_turn>user\nPlease act as an impartial judge and evaluate the quality of the responses provided by two\nAI assistants to the user question displayed below. You should choose the assistant that\nfollows the user’s instructions and answers the user’s question better. Your evaluation\nshould consider factors such as the helpfulness, relevance, accuracy, depth, creativity,\nand level of detail of their responses. Begin your evaluation by comparing the two\nresponses and provide a short explanation. Avoid any position biases and ensure that the\norder in which the responses were presented does not influence your decision. Do not allow\nthe length of the responses to influence your evaluation. Do not favor certain names of\nthe assistants. Be as objective as possible. After providing your explanation, output your\nfinal verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"\nif assistant B is better, and "[[C]]" for a tie.\n<|The Start of Conversation between a User and two Assistants|>\n### User:\nHow to initialize the classification head when I do transfer learning. For example, I have a pre-trained vision transformer on ImageNet, and now I want to finetune it on StanfordCars\n\n### Assistant A:\nWhen you want to initialize the classification head for transfer learning, you can follow these steps:\n\n  1. Load the pre-trained model: First, load the pre-trained vision transformer model on ImageNet. You can use popular deep learning libraries like PyTorch or TensorFlow to load the model.\n  2. Freeze the pre-trained layers: Next, you need to freeze the pre-trained layers of the model so that they don\'t get updated during the training process. This is because the pre-trained layers have learned useful features that can be reused for the new task.\n  3. Add a new classification head: Now, you need to add a new classification head to the model. The classification head is responsible for outputting the predicted class for a given input image. The size of the classification head depends on the number of classes in the new task. For example, if you are working on the StanfordCars dataset, which has 196 classes, you would need a classification head with 196 output neurons.\n  4. Initialize the classification head: Initialize the weights of the classification head randomly or using pre-trained weights. This is important because the initial weights can have a significant impact on the performance of the model.\n  5. Train the model: Finally, you can train the model on the new task using your labeled data. During training, only the weights of the classification head will be updated, while the weights of the frozen pre-trained layers will remain unchanged. This allows the model to learn task-specific features while leveraging the pre-trained features.\n\nNote that there are different ways to initialize the classification head, such as using Xavier initialization, Kaiming initialization, or He initialization. You can choose the initialization method that works best for your specific task and model architecture.\n\n### Assistant B:\nTo initialize the classification head when performing transfer learning, follow these steps:\n\n1. Load the pre-trained vision transformer model that was trained on ImageNet.\n2. Replace the existing classification head with a new one suitable for the StanfordCars dataset.\n3. Initialize the weights of the new classification head with random values or pre-trained weights if available.\n4. Freeze the weights of the vision transformer backbone to prevent them from being updated during training.\n5. Train the model on the StanfordCars dataset, fine-tuning the classification head while keeping the backbone frozen.\n\nHere\'s an example implementation in Python using PyTorch:\n\n```python\nimport torch\nimport torch.nn as nn\nimport torchvision.models as models\n\n# Step 1: Load pre-trained vision transformer model\nvision_transformer = models.vit_base_patch16_224(pretrained=True)\n\n# Step 2: Replace classification head\nnum_classes = 196  # Number of classes in StanfordCars dataset\nvision_transformer.head = nn.Linear(vision_transformer.head.in_features, num_classes)\n\n# Step 3: Initialize weights of new classification head\nnn.init.xavier_uniform_(vision_transformer.head.weight)\nnn.init.constant_(vision_transformer.head.bias, 0)\n\n# Step 4: Freeze the weights of the backbone\nfor param in vision_transformer.parameters():\n    param.requires_grad = False\n\n# Step 5: Train the model on StanfordCars dataset\n# ...\n```\n\nIn this example, we load the pre-trained vision transformer model using `torchvision.models.vit_base_patch16_224`. We then replace the `head` (classification head) of the model with a new linear layer (`nn.Linear`) that has the appropriate number of output classes for the StanfordCars dataset.\n\nAfter replacing the classification head, we initialize its weights using Xavier initialization (`nn.init.xavier_uniform_`) and set its biases to zero (`nn.init.constant_`).\n\nFinally, we freeze the weights of the vision transformer backbone by setting `requires_grad=False` for all parameters. This ensures that only the weights of the classification head will be updated during training on the StanfordCars dataset.\n\n### User:\nI want to do full finetuning\n\n### Assistant A:\nIf you want to do full finetuning of the pre-trained vision transformer on the StanfordCars dataset, you can follow these steps:\n\n  1. Load the pre-trained model: Load the pre-trained vision transformer model on ImageNet. You can use popular deep learning libraries like PyTorch or TensorFlow to load the model.\n  2. Remove the last classification layer: The pre-trained model typically has a last classification layer that is specific to the ImageNet dataset. To adapt the model to the StanfordCars dataset, you need to remove this layer.\n  3. Add a new classification head: Add a new classification head to the model. The classification head is responsible for outputting the predicted class for a given input image. The size of the classification head depends on the number of classes in the StanfordCars dataset, which has 196 classes. You can add a fully connected layer with 196 output neurons as the new classification head.\n  4. Initialize the classification head: Initialize the weights of the classification head randomly or using pre-trained weights. This is important because the initial weights can have a significant impact on the performance of the model.\n  5. Freeze the pre-trained layers: Freeze the pre-trained layers of the model so that they don\'t get updated during the training process. This is because the pre-trained layers have learned useful features that can be reused for the new task.\n  6. Train the model: Finally, you can train the model on the StanfordCars dataset using your labeled data. During training, all the weights of the model will be updated, including the weights of the frozen pre-trained layers. This allows the model to learn task-specific features while leveraging the pre-trained features.\n\nNote that when you do full finetuning, you need to use a lower learning rate and a smaller number of training epochs compared to transfer learning, because the model has already learned a lot of useful features from the pre-trained weights. You can experiment with different learning rates and training schedules to find the best hyperparameters for your specific task and model architecture.\n\n### Assistant B:\nIf you want to perform full finetuning, where both the classification head and the backbone of the vision transformer are updated during training on the StanfordCars dataset, you can omit freezing the backbone weights. Here\'s an updated version of the code:\n\n```python\nimport torch\nimport torch.nn as nn\nimport torchvision.models as models\n\n# Step 1: Load pre-trained vision transformer model\nvision_transformer = models.vit_base_patch16_224(pretrained=True)\n\n# Step 2: Replace classification head\nnum_classes = 196  # Number of classes in StanfordCars dataset\nvision_transformer.head = nn.Linear(vision_transformer.head.in_features, num_classes)\n\n# Step 3: Initialize weights of new classification head\nnn.init.xavier_uniform_(vision_transformer.head.weight)\nnn.init.constant_(vision_transformer.head.bias, 0)\n\n# Step 4: Train the model on StanfordCars dataset\n# ...\n```\n\nIn this case, you can directly train the vision transformer model on the StanfordCars dataset without freezing any weights. During training, both the weights of the classification head and the backbone will be updated.\n\nKeep in mind that full finetuning may require more computational resources and a larger labeled dataset compared to freezing the backbone. It can help the model learn more specific features related to the StanfordCars dataset but may also increase the risk of overfitting if the dataset is small. Monitoring the training progress and adjusting hyperparameters, such as learning rate and regularization, can help mitigate overfitting.\n<|The End of Conversation between a User and two Assistants|><end_of_turn>\n<start_of_turn>model\nverdict is: [[']}
```

About dataset manipulation:
```python
max_sequence_length = 4096 # after processing data tokenize with this length on text

# on dataloader
batch_size = 80
num_worker = 4
max_token = 1024*16 # when collate the text with the logic of Variable length Collator and this sequence will create micro_batch, which will have multiple sequence combined keeping in mind the tokens length. visialize it above below outputs

```


details about Data loader from train file have some this features mentioned below
```python

for batch in tqdm(dataloader):
    for micro_batch in batch:
        # print(micro_batch)
        print(len(micro_batch['input_ids'][0]))
        print(len(micro_batch['position_ids'][0]))
        print(micro_batch['cu_seqlens'])
        print(len(micro_batch['cu_seqlens']))
        print(len(micro_batch['input_text']))
        print(''.join(micro_batch['input_text'][0].split()[-10:])) # last 10 words
        print("------------------------------")

```


``` markdown
13316
13316
tensor([    0,  4330,  5251,  6259,  6564,  9577, 10569, 11543, 13001, 13316],
       dtype=torch.int32)
10
9
betweenaUserandtwoAssistants|><end_of_turn><start_of_turn>modelverdictis:[[
------------------------------
16007
16007
tensor([    0,  3097,  3392,  3836,  4784,  5131,  5468,  6478,  6849,  8687,
         8992,  9545, 10015, 10286, 11698, 12612, 13436, 14668, 16007],
       dtype=torch.int32)
19
18
betweenaUserandtwoAssistants|><end_of_turn><start_of_turn>modelverdictis:[[
------------------------------
15886
15886
tensor([    0,   922,  1470,  4863,  5648,  6447,  7456,  8115,  9301, 10276,
        10720, 11237, 12055, 12350, 13605, 13924, 14541, 15159, 15886],
       dtype=torch.int32)
19
18
betweenaUserandtwoAssistants|><end_of_turn><start_of_turn>modelverdictis:[[
------------------------------
4681
4681
tensor([   0,  500, 2058, 2876, 3201, 4681], dtype=torch.int32)
6
5
betweenaUserandtwoAssistants|><end_of_turn><start_of_turn>modelverdictis:[[
------------------------------
15993
15993
tensor([    0,   296,   676,  2148,  3009,  3411,  4666,  5568,  6410,  6974,
         7420,  7805,  8594,  9529, 10444, 10998, 11855, 12336, 14436, 14993,
        15993], dtype=torch.int32)
21
20
betweenaUserandtwoAssistants|><end_of_turn><start_of_turn>modelverdictis:[[
------------------------------
15929
15929
tensor([    0,   785,  1133,  2297,  3227,  3687,  4576,  5708,  6093,  6936,
         7347,  9561, 11896, 12671, 15228, 15929], dtype=torch.int32)
16
15
betweenaUserandtwoAssistants|><end_of_turn><start_of_turn>modelverdictis:[[
------------------------------
```

PS. you have script prepared to download the data_loader in https://www.kaggle.com/code/vanshikagupta1136/lmsys-0805/edit

**Model Building**

- Input will be for each micro_batch in batch `model(micro_batch["batch_size"], micro_batch["input_ids"], micro_batch["cu_seqlens"])`


**Model Config**

In [ ]:
# gemma2 model config

vocab_size=256000,
hidden_size=2304,
intermediate_size=9216,
num_hidden_layers=26,
num_attention_heads=8,
num_key_value_heads=4,
head_dim=256,
hidden_activation="gelu_pytorch_tanh",
max_position_embeddings=8192,
initializer_range=0.02,
rms_norm_eps=1e-6,
use_cache=True,
pad_token_id=0,
eos_token_id=1,
bos_token_id=2,
tie_word_embeddings=True,
rope_theta=10000.0,
attention_bias=False,
attention_dropout=0.0,
query_pre_attn_scalar=256,
sliding_window=4096,
final_logit_softcapping=30.0,
attn_logit_softcapping=50.0,
cache_implementation="hybrid",

**Gemma2PreTrainedModel**

- 2 modules mainly needs to be aware with is `Pretrainedmodel()` from transformers library and `nn.module` from pytorch library

In [ ]:
from torch import nn
import torch

base_model_prefix = "model"
supports_gradient_checkpointing = True
_no_split_modules = ["Gemma2DecoderLayer"]
_skip_keys_device_placement = ["past_key_values"]
_supports_flash_attn_2 = True
_supports_sdpa = True
_supports_cache_class = False
_supports_quantized_cache = False
_supports_static_cache = True
_is_stateful = True


# _init_weights for both type of module, in a gaussian distribution
# from gemma2config
std =  0.2 # initializer_range

# i added whichever module we have nn.Linear or nn.Embedding
module = nn.Linear(10, 30)
if isinstance(module, nn.Linear):
    module.weight.data.normal_(mean=0.0, std=std)
    if module.bias is not None:
        module.bias.data.zero_()

elif isinstance(module, nn.Embedding):
    module.weight.data.normal_(mean=0.0, std=std)
    if module.padding_idx is not None:
        module.weight.data[module.padding_idx].zero_()

module

TODO: TO run model till we get input_ids on real dataset setup

- Now Gemma2 model have 2 components which is using to design architecture
    - **Gemma2DecoderLayer** for multiple layers
    - **Gemma2RMSNorm** for normalization

In nn.Embedding class:

`padding_idx` (int, optional) – If specified, the entries at padding_idx do not contribute to the gradient; therefore, the embedding vector at padding_idx is not updated during training, i.e. it remains as a fixed “pad”. For a newly constructed Embedding, the embedding vector at padding_idx will default to all zeros, but can be updated to another value to be used as the padding vector.

- Benefit: Padding tokens are often used to ensure that all sequences in a batch have the same length. By specifying padding_idx, the embedding layer can ignore these padding tokens during the forward pass, which can lead to computational savings.

In [ ]:
import torch
from torch import nn

# for creating sample model
input_id = torch.randint(0, 10000, (1020,)) # in model design 16 * 1024 is max of sequence of tokens # (T)
input_id = input_id.unsqueeze(0)
print(input_id)

vocab_size = 256000 # from gemma2 config
padding_idx = 0 # from gemma2 config
hidden_size = 2304

input_id = input_id.squeeze(0)
print(input_id)
embed_tokens = nn.Embedding(num_embeddings = vocab_size, embedding_dim = hidden_size, padding_idx = padding_idx) #(V,C)
inputs_embeds = embed_tokens(input_id)
print(inputs_embeds) # (T,C)

In [ ]:
print(inputs_embeds.shape) #(T, C)

**Embedding Normalized**

In [ ]:
hidden_states = inputs_embeds

print(hidden_size**0.5)
normalizer = torch.tensor(hidden_size**0.5, dtype=hidden_states.dtype)
hidden_states = hidden_states * normalizer
normalizer

**Rotary Embedding**

In [ ]:
import torch
head_dim = 256
max_position_embeddings = 8192
dim = head_dim
inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
seq = torch.arange(max_position_embeddings, dtype=inv_freq.dtype)
freqs = torch.einsum("i , j -> i j", seq, inv_freq)
emb = torch.cat((freqs, freqs), dim=-1)
emb = emb.reshape(emb.size(0), 1, 1, emb.size(1))
rotary_emb = emb
print(rotary_emb)
print(rotary_emb.shape)
print(len(rotary_emb))

- simple implementation in transformer gemma2 model

```python
query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

# position_embeddings # from Gemma2RotaryEmbedding
# """Applies Rotary Position Embedding to the query and key tensors.

# Args:
#     q (`torch.Tensor`): The query tensor.
#     k (`torch.Tensor`): The key tensor.
#     cos (`torch.Tensor`): The cosine part of the rotary embedding.
#     sin (`torch.Tensor`): The sine part of the rotary embedding.
#     position_ids (`torch.Tensor`, *optional*):
#         Deprecated and unused.
#     unsqueeze_dim (`int`, *optional*, defaults to 1):
#         The 'unsqueeze_dim' argument specifies the dimension along which to unsqueeze cos[position_ids] and
#         sin[position_ids] so that they can be properly broadcasted to the dimensions of q and k. For example, note
#         that cos[position_ids] and sin[position_ids] have the shape [batch_size, seq_len, head_dim]. Then, if q and
#         k have the shape [batch_size, heads, seq_len, head_dim], then setting unsqueeze_dim=1 makes
#         cos[position_ids] and sin[position_ids] broadcastable to the shapes of q and k. Similarly, if q and k have
#         the shape [batch_size, seq_len, heads, head_dim], then set unsqueeze_dim=2.
# Returns:
#     `tuple(torch.Tensor)` comprising of the query and key tensors rotated using the Rotary Position Embedding.
# """

# cos, sin = position_embeddings

# q, k, cos, sin, position_ids=None, unsqueeze_dim=1
# cos = cos.unsqueeze(unsqueeze_dim)
# sin = sin.unsqueeze(unsqueeze_dim)
# q_embed = (q * cos) + (rotate_half(q) * sin)
# k_embed = (k * cos) + (rotate_half(k) * sin)

```
- optimized implementation in transformer-engine gemma2 model



```python
query_states = te.attention.FusedRoPEFunc.apply(
            query_states, rotary_emb, "thd", cu_seqlens
        )
        key_states = te.attention.FusedRoPEFunc.apply(
            key_states, rotary_emb, "thd", cu_seqlens
        )
```

- [RoPE: From Understanding to Implementation](https://zhuanlan.zhihu.com/p/17797971938)
- Transformer upgrade path: 2. Rotary position encoding : https://spaces.ac.cn/archives/8265
- video [How Rotary Position Embedding Supercharges Modern LLMs](https://www.youtube.com/watch?v=SMBkImDWOyQ) good explanations and blogs : [Rotary Position Encoding](https://aakashkumarnain.github.io/posts/ml_dl_concepts/rope)

**Gemma2DecoderLayer**
- Gemma2SdpaAttention
- Gemma2MLP
- Gemma2RGMSNorm

**Gemma2Attention**

<!-- ![image.png](attachment:d2278852-3bc1-475f-9787-85c107abf9ab.png) -->
chekc out attention image from gemma2 model for attentions

**Multi Query Attention Head**
- Blog [Multi Query Attention is all you need](https://fireworks.ai/blog/multi-query-attention-is-all-you-need)

-


In [ ]:
num_hidden_layers=26
is_last_decoder_layer = False
head_dim=256 # 32 * 4 * 2
attention_bias = False
num_heads = 8
num_key_value_heads = 4
hidden_size = 2304

q_len, _ = hidden_states.size() # T
# q_proj :
q_proj = nn.Linear(in_features = hidden_size, out_features = num_heads * head_dim, bias = attention_bias)
k_proj = nn.Linear(in_features = hidden_size, out_features = num_key_value_heads * head_dim, bias = attention_bias)
v_proj = nn.Linear(in_features = hidden_size, out_features = num_key_value_heads * head_dim, bias = attention_bias)
o_proj = nn.Linear(in_features = hidden_size, out_features = num_heads * head_dim, bias = attention_bias)

print(f"q_proj : {q_proj}, k_proj : {k_proj}, v_proj : {v_proj}, o_proj : {o_proj}\n\n")

query_states = q_proj(hidden_states)
key_states = k_proj(hidden_states)
value_states = v_proj(hidden_states)

# T, head*dim   | T, kvhead*dim  |  T, kvhead*dim
print(f"query_states : {query_states.shape}, key_states : {key_states.shape}, value_states : {value_states.shape}\n\n")


query_states = query_states.view(q_len, num_heads, head_dim)
key_states = key_states.view(q_len, num_key_value_heads, head_dim)
value_states = value_states.view(q_len, num_key_value_heads, head_dim)

print(f"query_states : {query_states.shape}, key_states : {key_states.shape}, value_states : {value_states.shape}\n\n")
# print(f"query_states : {query_states}, key_states : {key_states}, value_states : {value_states}\n\n")

I am gonna complete all layers of this decoder layers code today night

**Flash Attention**

- [Packing with Flash Attention for Efficient Fine-tuning of LLM](https://ho.lc/blog/packing_with_flash_attention)
    - contain concepts of flash attentions & varibale length

In [ ]:
# Above i considered layer_index = 1 for understanding
# for i in range(num_hidden_layers):

# Gemma2Decoder > P Rotation > Attention > MLP > RMSNorm

**Configuration to Run and Evaluate Model**

**FSDP**

 1. **FSDP (Fully Sharded Data Parallel)**
FSDP is a PyTorch module designed to scale model training across multiple GPUs by sharding (splitting) the model parameters and gradients across devices. This approach helps in efficiently using memory and computation, enabling the training of very large models that would otherwise not fit into a single GPU's memory.

 2. **Parameters of FSDP**

 **`model`**
This is the neural network model you want to wrap with FSDP. It can be any PyTorch `nn.Module`. When you pass this model to FSDP, FSDP will take care of sharding the model's parameters and gradients across the available GPUs.

 **`auto_wrap_policy`**
This parameter specifies the policy for automatically wrapping layers within the model with FSDP. The `auto_wrap_policy` can be a function or a predefined policy that determines which layers should be wrapped with FSDP. This is useful for large models where you want to shard only certain layers to optimize memory usage and performance.

- **How it works**: FSDP can automatically wrap sub-modules of the model based on certain criteria (e.g., module type, parameter count). This allows for fine-grained control over which parts of the model are sharded. For example, you might want to shard large layers like transformers but not small layers like batch normalization.
    

 **`sharding_strategy`**

This parameter specifies the strategy for sharding the model parameters. PyTorch provides several sharding strategies:

- **`ShardStrategy.FULL_SHARD`**: Shards the entire model across all devices. This means that each GPU will hold a portion of the model's parameters.
- **`ShardStrategy.SHARD_GRAD_OP`**: Shards only the gradients and optimizer states. This can be useful for reducing memory usage during the backward pass.
- **`ShardStrategy.NO_SHARD`**: Does not shard the model parameters. This is useful for debugging or when you want to control sharding manually.
- **How it works**: The chosen strategy determines how the model's parameters and gradients are distributed across the GPUs. For example, with `FULL_SHARD`, each GPU will only store a subset of the model's parameters, reducing the memory footprint on each device.
    

 **`device_id`**
This parameter specifies the device (GPU) on which the model will be placed. `torch.cuda.current_device()` returns the index of the current CUDA device. This ensures that the model is placed on the correct GPU in a multi-GPU setup.
- **How it works**: In a distributed training setup, each process (rank) will typically be assigned to a specific GPU. By specifying `device_id`, you ensure that the model is correctly placed on the intended GPU, allowing for efficient data transfer and computation.
    

 **`mixed_precision`**
This parameter enables mixed precision training, which combines single-precision (FP32) and half-precision (FP16) floating-point formats to reduce memory usage and improve training speed. Mixed precision training can significantly speed up the training process while maintaining model accuracy.

- **How it works**: Mixed precision training uses FP16 for most computations to save memory and speed up calculations, while critical operations (like gradient updates) are performed in FP32 to maintain numerical stability. This is typically managed using PyTorch's `torch.cuda.amp` module, which provides tools for automatic mixed precision training.
    

When you wrap a model with FSDP using this code snippet, the following happens:

1. **Model Initialization**: The model is passed to FSDP, which prepares it for distributed training.
2. **Auto Wrap Policy**: FSDP uses the specified `auto_wrap_policy` to determine which layers of the model should be wrapped and sharded across GPUs.
3. **Sharding Strategy**: The chosen `sharding_strategy` dictates how the model's parameters and gradients are distributed across the GPUs.
4. **Device Placement**: The model is placed on the specified `device_id`, ensuring it runs on the correct GPU.
5. **Mixed Precision**: Mixed precision training is enabled, allowing the model to use both FP16 and FP32 formats to optimize memory usage and training speed.
    

This setup is particularly useful for training large models efficiently across multiple GPUs, reducing memory usage, and improving training speed.

Detailing: https://pytorch.org/docs/stable/fsdp.html

In [ ]:
# model = FSDP(
#     model,
#     auto_wrap_policy=fsdp.auto_wrap_policy,
#     sharding_strategy=fsdp.sharding_strategy,
#     device_id=torch.cuda.current_device(),
#     mixed_precision=fsdp.mixed_precision
# )

Why We Use Gradient Clipping in FSDP

*   



**About** `dist.barrier()`
*


About GPUs architecture :
- Cornell [workshop](https://cvw.cac.cornell.edu/gpu-architecture/gpu-example-tesla-v100/index)
- Comparison blogs: [Nvdia T4 vs A100](https://www.server-parts.eu/post/nvidia-t4-vs-a100-gpu-comparison-ai-deep-learning-data-centers)


How many flops and how decide to choose GPU for learning

